In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial_2'
log_name = 'test'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 13:43:28.469233+00:00,Joe,0,1,COMPLETE,2020-01-01 14:59:41.608354+00:00,2,0 days 01:16:13.139121,4573,49408,1,0,0,0,0,0,1,0
1,REPAIR,START,2020-01-01 14:59:41.608354+00:00,Jane,0,4,COMPLETE,2020-01-04 18:38:11.639016+00:00,5,3 days 03:38:30.030662,272310,53981,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 19:33:48.900814+00:00,1,1,10,COMPLETE,2020-01-01 20:34:39.406353+00:00,11,0 days 01:00:50.505539,3650,70428,1,0,0,1,0,0,0,0
4,REPAIR,START,2020-01-01 20:34:39.406353+00:00,1,1,13,COMPLETE,2020-01-02 09:05:17.659036+00:00,14,0 days 12:30:38.252683,45038,74079,1,0,1,2,0,0,0,0
6,DIAGNOSIS,START,2020-01-02 07:47:21.536525+00:00,Clark,2,19,COMPLETE,2020-01-02 08:45:47.275476+00:00,20,0 days 00:58:25.738951,3505,28041,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,QUALITY_CONTROL,START,2022-06-30 11:11:14.473595+00:00,Jane,890,8017,COMPLETE,2022-06-30 13:45:04.355487+00:00,8018,0 days 02:33:49.881892,9229,40274,1,1,1,0,0,3,0,0
2669,QUALITY_CONTROL,START,2022-06-30 12:13:46.487067+00:00,Joe,889,8008,COMPLETE,2022-06-30 19:40:59.136223+00:00,8009,0 days 07:27:12.649156,26832,44026,1,1,1,1,0,1,1,0
2673,DIAGNOSIS,START,2022-06-30 22:25:05.097535+00:00,Karsten,891,8020,COMPLETE,2022-06-30 23:15:20.606227+00:00,8021,0 days 00:50:15.508692,3015,80705,1,0,0,0,0,0,0,1
2674,REPAIR,START,2022-06-30 23:15:20.606227+00:00,Jane,891,8023,COMPLETE,2022-07-01 07:28:48.982352+00:00,8024,0 days 08:13:28.376125,29608,83720,1,0,1,0,0,1,0,1


In [4]:
N = 10000
n_processes = 10
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                               | 1/892 [01:26<21:23:33, 86.43s/it]

  0%|▏                                                                               | 2/892 [01:26<8:51:38, 35.84s/it]

  1%|▍                                                                               | 5/892 [01:28<2:36:54, 10.61s/it]

  1%|▌                                                                               | 6/892 [01:29<1:58:53,  8.05s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 879/892 [01:44<00:01, 10.46it/s]

 99%|███████████████████████████████████████████████████████████████████████████████ | 881/892 [01:45<00:01,  6.55it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▎| 885/892 [01:45<00:00,  8.17it/s]

100%|███████████████████████████████████████████████████████████████████████████████▉| 891/892 [01:45<00:00, 11.76it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00,  8.42it/s]

In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.01054175110184322365377255197')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(13160.3401381431)

In [8]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                               | 1/892 [01:18<19:21:47, 78.23s/it]

  0%|▎                                                                               | 4/892 [01:20<3:49:02, 15.48s/it]

  1%|▉                                                                              | 10/892 [01:22<1:11:26,  4.86s/it]

  1%|▉                                                                              | 11/892 [02:20<3:03:20, 12.49s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▍| 886/892 [01:44<00:00, 10.14it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 888/892 [01:44<00:00,  9.98it/s]

100%|███████████████████████████████████████████████████████████████████████████████▊| 890/892 [01:45<00:00,  8.44it/s]

100%|███████████████████████████████████████████████████████████████████████████████▉| 891/892 [01:45<00:00,  7.62it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:45<00:00,  8.47it/s]

In [9]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('0.2545875417511894589575843913')

In [10]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(10956.32406107799)

In [11]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                               | 1/892 [01:37<24:04:51, 97.30s/it]

  0%|▏                                                                              | 2/892 [01:38<10:08:38, 41.03s/it]

  0%|▎                                                                               | 4/892 [01:39<3:50:01, 15.54s/it]

  1%|▋                                                                               | 7/892 [01:40<1:39:49,  6.77s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 884/892 [01:20<00:00, 11.37it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▍| 886/892 [01:20<00:00, 10.20it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 888/892 [01:20<00:00, 11.20it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:20<00:00, 11.77it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:20<00:00, 11.05it/s]

In [12]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

Decimal('-0.2436209489321157003557800197')

In [13]:
np.mean(get_pscores(likelihoods_R_S))

np.float64(20503.570113905447)

In [14]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                               | 1/892 [01:04<15:54:04, 64.25s/it]

  0%|▏                                                                              | 2/892 [01:29<10:09:40, 41.10s/it]

  1%|▊                                                                               | 9/892 [01:29<1:28:23,  6.01s/it]

  1%|▊                                                                               | 9/892 [01:41<1:28:23,  6.01s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 885/892 [01:20<00:00, 12.62it/s]

 99%|███████████████████████████████████████████████████████████████████████████████▌| 887/892 [01:21<00:00, 12.07it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 889/892 [01:21<00:00,  8.54it/s]

100%|███████████████████████████████████████████████████████████████████████████████▉| 891/892 [01:21<00:00,  9.90it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [01:21<00:00, 10.93it/s]

In [15]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

Decimal('-0.1472932607871942772457732751')

In [16]:
np.mean(get_pscores(likelihoods_R))

np.float64(18112.801652090482)

In [17]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                               | 1/892 [00:56<13:52:54, 56.09s/it]

  0%|▏                                                                               | 2/892 [00:56<5:48:34, 23.50s/it]

  0%|▎                                                                               | 3/892 [00:57<3:12:20, 12.98s/it]

  1%|▍                                                                               | 5/892 [00:57<1:25:48,  5.80s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████▊ | 879/892 [00:48<00:00, 20.39it/s]

 99%|███████████████████████████████████████████████████████████████████████████████ | 882/892 [00:48<00:00, 16.54it/s]

100%|███████████████████████████████████████████████████████████████████████████████▋| 889/892 [00:48<00:00, 20.58it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [00:49<00:00, 16.57it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [00:49<00:00, 18.19it/s]

In [18]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

Decimal('0.1827566423670557178817743735')

In [19]:
np.mean(get_pscores(likelihoods_A_R_S))

np.float64(12534.721241238378)

In [20]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

  0%|                                                                                          | 0/892 [00:00<?, ?it/s]

  0%|                                                                                | 1/892 [00:26<6:36:03, 26.67s/it]

  0%|▏                                                                               | 2/892 [00:26<2:44:00, 11.06s/it]

  0%|▎                                                                               | 3/892 [00:26<1:29:53,  6.07s/it]

  1%|▍                                                                                 | 5/892 [00:27<39:22,  2.66s/it]

...

 99%|██████████████████████████████████████████████████▎| 881/892 [00:59<00:00, 12.54it/s]

 99%|██████████████████████████████████████████████████▍| 883/892 [00:59<00:00, 10.58it/s]

100%|██████████████████████████████████████████████████▊| 888/892 [01:00<00:00, 14.70it/s]

100%|███████████████████████████████████████████████████| 892/892 [01:00<00:00, 16.00it/s]

100%|███████████████████████████████████████████████████| 892/892 [01:00<00:00, 14.78it/s]

In [21]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

Decimal('-0.1710989190580732570881604812')

In [22]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

np.float64(10508.593165846229)

In [23]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|                                                             | 0/892 [00:00<?, ?it/s]

  0%|                                                   | 1/892 [00:27<6:55:21, 27.97s/it]

  0%|                                                   | 2/892 [00:28<2:52:43, 11.64s/it]

  0%|▏                                                  | 3/892 [00:45<3:33:31, 14.41s/it]

  1%|▍                                                    | 7/892 [00:46<59:35,  4.04s/it]

...

 98%|██████████████████████████████████████████████████▏| 878/892 [00:59<00:00, 14.69it/s]

 99%|██████████████████████████████████████████████████▍| 882/892 [00:59<00:00, 14.82it/s]

100%|██████████████████████████████████████████████████▊| 888/892 [01:00<00:00, 18.80it/s]

100%|██████████████████████████████████████████████████▉| 891/892 [01:00<00:00, 18.63it/s]

100%|███████████████████████████████████████████████████| 892/892 [01:00<00:00, 14.78it/s]

In [24]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

Decimal('0.3098812832393363862910922221')

In [25]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

np.float64(10804.230970171016)

In [26]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|                                                             | 0/892 [00:00<?, ?it/s]

  0%|                                                   | 1/892 [00:24<6:06:03, 24.65s/it]

  0%|▏                                                  | 3/892 [00:34<2:31:08, 10.20s/it]

  1%|▎                                                  | 6/892 [00:44<1:27:28,  5.92s/it]

  1%|▌                                                 | 11/892 [01:09<1:17:58,  5.31s/it]

...

 99%|██████████████████████████████████████████████████▎| 879/892 [00:56<00:00, 18.57it/s]

 99%|██████████████████████████████████████████████████▍| 882/892 [00:56<00:00, 19.88it/s]

 99%|██████████████████████████████████████████████████▌| 885/892 [00:56<00:00, 17.28it/s]

100%|██████████████████████████████████████████████████▊| 888/892 [00:57<00:00, 14.62it/s]

100%|███████████████████████████████████████████████████| 892/892 [00:57<00:00, 15.56it/s]

In [27]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

Decimal('0.1943068918954015224536706409')

In [28]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

np.float64(12997.949072439838)

In [29]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)